In [1]:
import os, random, time, pymysql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import openpyxl
from scipy.stats import mode

In [2]:
f_list = os.listdir('./data/')
f_list.sort()
production_file = [i for i, v in enumerate(f_list) if 'PRODUCTION' in v.split('_')]
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
inven_file = [i for i, v in enumerate(f_list) if 'INVENTORY' in v.split('_')]
print(f_list)
print(production_file, '\n', sale_file, '\n', inven_file)

['1_DMF0035_MktFcstWaterfall(Mkt Fcst Waterfall)_201901_202003.xlsx', '2_DMF0035_MktFcstWaterfall(Mkt Fcst Waterfall)_201909_202112.xlsx', '3_DAIMLER_E61D_SCP_PRODUCTION_PLANNING_v0512.xlsx', '4_DAIMLER_E61D_SCP_PRODUCTION_RESULT_2018_10_2021_03_v2.xlsx', '5_Daimler_E61D_INVENTORY_210512_v3.xlsx']
[2, 3] 
 [0, 1] 
 [4]


In [3]:
def dta_merge(t_tmp, t_title):
    #t_tmp = tmp1 ; t_title = tmp1.columns
    ver_col = np.where(t_tmp.columns == 'Version')[0][0]
    t_tmp1 = pd.DataFrame(t_tmp.iloc[:,ver_col].copy())
    t_tmp1 = pd.concat([t_tmp1,t_tmp.iloc[:, 14:]], axis=1)
    return t_tmp1
def year_week_cal(tmp_set_, col_name_, format_):
    tmp_cal_year = list(pd.to_datetime(tmp_set_[col_name_], format=format_).dt.isocalendar().year.values)
    tmp_cal_ww   = pd.to_datetime(tmp_set_[col_name_], format=format_).dt.isocalendar().week.values
    tmp_cal_ww1 = list(map(lambda x : str(tmp_cal_ww[x]) if divmod(tmp_cal_ww[x], 10)[0] > 0 else str(0)+str(tmp_cal_ww[x]), range(0, len(tmp_cal_year))))
    tmp_cal_date = list(map(lambda x : str(tmp_cal_year[x]) + tmp_cal_ww1[x], range(0, len(tmp_cal_year))))
    return tmp_cal_date

## Inventory Dataset

In [6]:
for i in range(0, len(inven_file)):
    tmp_file_raw = openpyxl.load_workbook('./data/'+f_list[inven_file[i]])
    sheet_name = '02. 일별 PSI 트렌드' #tmp_file_raw.sheetnames
    tmp_file_pd = pd.DataFrame(tmp_file_raw[sheet_name].values).copy()
    tmp_col_name1 = list(tmp_file_pd.iloc[0,0:])
    tmp_file = tmp_file_pd.iloc[1:,:].copy().reset_index(drop=True)
    tmp_file.columns = tmp_col_name1
    tmp_day = pd.to_datetime(tmp_file['연월일'], format='%Y년%m월%d일').dt.isocalendar().day
    tmp_file1 = tmp_file[(tmp_file['자재'].str.slice(0,9) == 'ACEN1060I') & (tmp_day == 7)].copy()
    inven_filter_ = [['3', 'C'], ['3','T'],['5','0']]
    for k in range(0, len(inven_filter_)):
        tmp_store1, tmp_store2 = inven_filter_[k][0], inven_filter_[k][1]
        tmp_file1_sub1 = tmp_file1[(tmp_file1['저장위치'].str.slice(0,1) == tmp_store1)&(tmp_file1['저장위치'].str.slice(3,4) == tmp_store2)].copy()
        tmp_week_ = year_week_cal(tmp_file1_sub1, '연월일', '%Y년%m월%d일')
        tmp_file1_sub1['Inven Week'] = tmp_week_        
        tmp_file1_sub1 = tmp_file1_sub1[['Inven Week','연월일', '재고수량']]
        tmp_group = tmp_file1_sub1.groupby(['Inven Week'])
        tmp_sum = pd.DataFrame(tmp_group.sum())
        tmp_sum.reset_index(inplace=True)    
        tmp_sum.to_csv('./data1/Inven_%sxx%s.csv'%(tmp_store1, tmp_store2), index=False)
        print('Save Complete : Inven_%sxx%s.csv'%(tmp_store1, tmp_store2))

Save Complete : Inven_3xxC.csv
Save Complete : Inven_3xxT.csv
Save Complete : Inven_5xx0.csv


## Product Dataset

In [7]:
for i in range(0, len(production_file)): #1):#    
    tmp_file_raw = openpyxl.load_workbook('./data/'+f_list[production_file[i]])
    print("Load Product %s File"%f_list[production_file[i]])
    if 'RESULT' in f_list[production_file[i]].split('_'):   
        sheet_name = '02.Detail_생산계획 준수율'
        tmp_file_pd = pd.DataFrame(tmp_file_raw[sheet_name].values).copy()
        tmp_col_name1 = list(tmp_file_pd.iloc[5,0:])
        tmp_file = tmp_file_pd.iloc[6:, :].copy().reset_index(drop=True)
        tmp_file.columns = tmp_col_name1
        tmp_file1 = tmp_file[(tmp_file['Material'].str.slice(0,9)=='ACEN1060I') & (tmp_file['Nick Name'].str.slice(0,4)=='E61D')].copy().reset_index(drop=True)
        tmp_file1 = tmp_file1[['Plan Date', 'Basic Date', 'Plan Week', 'Result Qty']] #'Plan Date', 
        tmp_file1['WW'] = pd.to_datetime(tmp_file1['Plan Date']).dt.isocalendar().week
        tmp_file1 = tmp_file1[['Plan Week', 'Basic Date', 'WW', 'Result Qty']] #'Plan Date',         
        tmp_group = tmp_file1.groupby(['Plan Week', 'WW', 'Basic Date'])
        tmp_g_sum = pd.DataFrame(tmp_group.sum())
        tmp_g_sum.reset_index(inplace=True)
        tmp_g_cnt = pd.DataFrame(tmp_group.count())
        tmp_g_cnt.reset_index(inplace=True)
        tmp_g = pd.merge(tmp_g_sum, tmp_g_cnt, on=['Plan Week', 'WW', 'Basic Date'])
    else:
        tmp_file_raw = openpyxl.load_workbook('./data/'+f_list[production_file[i]])
        sheet_name = 'DynamicResult'
        tmp_file_pd = pd.DataFrame(tmp_file_raw[sheet_name].values).copy()
        tmp_col_name1 = list(tmp_file_pd.iloc[0,0:])
        tmp_file = tmp_file_pd.iloc[1:, :].copy().reset_index(drop=True)
        tmp_file.columns = tmp_col_name1
        tmp_file1 = tmp_file[['PLAN_START_YYYYMMDD', 'YYYYMMDD', 'QTY']].copy() #'Plan Date', 
        tmp_plan_date_= year_week_cal(tmp_file1, 'PLAN_START_YYYYMMDD', '%Y%m%d')
        tmp_date_= year_week_cal(tmp_file1, 'YYYYMMDD', '%Y%m%d')
        tmp_file1['Plan Date'] = tmp_plan_date_
        tmp_file1['Date']=tmp_date_
        tmp_file1 = tmp_file1[['Plan Date', 'PLAN_START_YYYYMMDD','Date', 'QTY']] #'PLAN_START_YYYYMMDD', 'YYYYMMDD', 
        tmp_group = tmp_file1.groupby(['Plan Date', 'PLAN_START_YYYYMMDD', 'Date'])
        tmp_g_sum = pd.DataFrame(tmp_group.sum())
        tmp_g_sum.reset_index(inplace=True)
        tmp_g_cnt = pd.DataFrame(tmp_group.count())
        tmp_g_cnt.reset_index(inplace=True)
        tmp_g = pd.merge(tmp_g_sum, tmp_g_cnt, on=['Plan Date', 'Date', 'PLAN_START_YYYYMMDD'])
    tmp_g.to_csv('./data1/'+sheet_name+".csv", index=False)

Load Product 3_DAIMLER_E61D_SCP_PRODUCTION_PLANNING_v0512.xlsx File
Load Product 4_DAIMLER_E61D_SCP_PRODUCTION_RESULT_2018_10_2021_03_v2.xlsx File


## Sales Dataset

In [25]:
for f in range(0, len(sale_file)):
    print(f_list[sale_file[f]])
    tmp_file_raw = openpyxl.load_workbook('./data/'+f_list[sale_file[f]])
    sheet_name = 'Mkt Fcst Waterfall'
    tmp_file = pd.DataFrame(tmp_file_raw[sheet_name].values)
    
    ind_col = np.where(tmp_file.iloc[0, :] == 'Measure')[0][0]
    col_name1 = list(tmp_file.iloc[0,:(ind_col+1)])
    tmp_col_name2 = list(tmp_file.iloc[0,(ind_col+1):])
    #tmp_col_name2 = [x.strftime('%Y-%m') for x in tmp_col_name2]
    tmp_col_name2_ = tmp_col_name2
    tmp_col_name3 = list(tmp_file.iloc[1,(ind_col+1):])
    col_name2 = []
    for i in range(0, len(tmp_col_name3)):
        if tmp_col_name3[i] == "SUM":
            col_name2.append(tmp_col_name3[i])
        else:
            if tmp_col_name2_[i][5:] == '12' and tmp_col_name3[i][1:] == '01':
                col_name2.append(str(int(tmp_col_name2_[i][:4])+1)+tmp_col_name3[i][1:])
            elif tmp_col_name2_[i][5:] == '01' and tmp_col_name3[i][1:] == '53':
                col_name2.append(str(int(tmp_col_name2_[i][:4])-1)+tmp_col_name3[i][1:])                
            else:
                col_name2.append(tmp_col_name2_[i][:4]+tmp_col_name3[i][1:])
    col_name = col_name1 + col_name2
    tmp_file1 = tmp_file.iloc[3:,:].reset_index(drop=True).copy()
    tmp_file1.columns = col_name
    #tmp_file1 = tmp_file1[tmp_file1['Model'].str.slice(0,9)=='APVCCCMA0'].copy().reset_index(drop=True)
    if 'SUM' in col_name:
        rm_col1 = np.array([i for i, val in enumerate(col_name) if val == 'SUM'])
        tmp_file1.drop(tmp_file1.iloc[:, rm_col1], axis=1, inplace=True)
    keep_col_name = list(['Version']) + list(set(tmp_file1.columns[(ind_col+1):])) #list(['Version']) + list(tmp_file1.columns[(ind_col+1):])
    tmp_file1 = tmp_file1[keep_col_name].copy()    
    tmp_file1_title = tmp_file1.columns.unique()
    tmp_file2_val = tmp_file1['Version'].copy()
    for k in range(1, len(tmp_file1_title)):
        tmp_i = [i for i, val in enumerate(tmp_file1.columns) if val == tmp_file1_title[k] ]
        #print(k, tmp_i, tmp_file1_title[k])
        if len(tmp_i) > 1:
            #tmp_file2.append(list(())
            tmp_file2_val = pd.concat([tmp_file2_val, tmp_file1.iloc[:,tmp_i].fillna(0).sum(axis=1)], axis=1)
        else :
            tmp_file2_val = pd.concat([tmp_file2_val, tmp_file1.iloc[:,tmp_i].fillna(0)], axis=1)
    tmp_file2_val.columns = tmp_file1_title
    if any(tmp_file2_val['Version'] == 'CURRENT'):
        tmp_file2_val = tmp_file2_val[tmp_file2_val['Version'] != 'CURRENT'] 
        
    tmp_file2 = pd.DataFrame({'Date' : year_week_cal(tmp_file2_val, 'Version', 'MF_W_%Y%m%d_V001')})
    tmp_file2 = pd.concat([tmp_file2, tmp_file2_val.drop(['Version'], axis=1)], axis=1)
    tmp_file2 = tmp_file2.sort_values(by=['Date'], axis=0)
    tmp_group = tmp_file2.groupby('Date')
    tmp_g_sum = pd.DataFrame(tmp_group.sum())
    tmp_g_sum = tmp_g_sum.sort_index(axis=1)
    tmp_g_sum.reset_index(inplace=True)
    tmp_g_sum.to_csv('./data1/'+f_list[sale_file[f]][2:].split('.')[0]+'.csv', index=False)


1_Volvo_MktFcstWaterfall(Mkt Fcst Waterfall)_201901-202003.xlsx
2_Volvo_MktFcstWaterfall(Mkt Fcst Waterfall)_201909-202112.xlsx
